# Chapter 2
## Multi-armed Bandits

### 2.2 Action-value Methods

#### Exercise 2.1
In $\epsilon$-greedy action selection, for the case of two actions and $\epsilon$ = 0.5, what is
the probability that the greedy action is selected?

### 2.3 The 10-armed Testbed

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

from bandit import run_episode
from agents import EpsilonGreedyAgent


k = 10
seed = 42
num_envs = 2000

env = gym.make_vec("ArmedBandit-v0", num_envs, "custom", vector_kwargs={"k": k})
env.reset(seed=seed)
env.render()

In [ ]:
agent = EpsilonGreedyAgent(k, 0.0, num_envs)
agent.reset(seed=seed)

rewards = run_episode(env, agent)

plt.plot(rewards.mean(axis=1))
plt.show()